In [10]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go

In [11]:
site = 2
site_label  = 'PA'

In [12]:
cfg = load_yaml('cfg.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

In [13]:
def results(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{experiments[exp_i]}'

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{experiments[exp_i]}.data'

        result_df = pd.read_pickle(final_results_file)
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'Global',
                'metric': 'Precision',
                'value': result_df['global_precision'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'Global',
                'metric': 'Recall',
                'value': result_df['global_recall'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'Global',
                'metric': 'F1-Score',
                'value': result_df['global_f1'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'No Cloud',
                'metric': 'Precision',
                'value': result_df['no_cloud_precision'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'No Cloud',
                'metric': 'Recall',
                'value': result_df['no_cloud_recall'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'No Cloud',
                'metric': 'F1-Score',
                'value': result_df['no_cloud_f1'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'Cloudy',
                'metric': 'Precision',
                'value': result_df['cloud_precision'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'Cloudy',
                'metric': 'Recall',
                'value': result_df['cloud_recall'][0],
            }
        )
        exp_results.append(
            {
                'name': exp_i,
                'Cloud Condition': 'Cloudy',
                'metric': 'F1-Score',
                'value': result_df['cloud_f1'][0],
            }
        )
        
        
    return pd.DataFrame(exp_results)


In [20]:
experiments = {
    'Opt (Diverse Cond)': 1,
    'Opt (Cloud-free)': 6,
    'SAR (12 Avg Images)': 2,
    'Early Fusion (Diverse Opt + 12 Avg SAR)': 3,
    'Joint Fusion (Diverse Opt + 12 Avg SAR)': 4,
    'Late Fusion (Diverse Opt + 12 Avg SAR)': 5,
    'Joint Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 11,
    'Late Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 12,
}
pixel_cond = 'Global'
#pixel_cond = 'Cloudy'
#pixel_cond = 'No Cloud'
exp_results = results(experiments)
#exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
exp_results = exp_results.loc[exp_results['Cloud Condition'] == pixel_cond]
fig = px.bar(exp_results, x = 'metric', y= 'value', color='name', barmode='group', text='value', facet_col = 'Cloud Condition', width=1400, height=400)
#fig.update_xaxes(categoryorder='array', categoryarray= ['No Cloud', 'Cloudy', 'Global'])
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1.05], title_text = '', tickformat = '.1f') 
fig.update_xaxes(
    title_text = '',
    tickfont_size = 20
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    legend_orientation = 'h',
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label} ({pixel_cond})',
    title_x = 0.5
    )
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [15]:
experiments = {
    'SAR (12 Avg Images)': 2,
    'SAR (2 Avg Images)': 7,
    'SAR (2 Single Images)': 8,
    'SAR (All Single Images)': 9,
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    )

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [16]:
experiments = {
    'Opt (WBCE, ignore class 2)': '6_wbce_2_8_adamw',
    'Opt (Focal gamma=3, ignore class 2)': '6_gamma3-lr-2e-5',
    'Opt (Focal gamma=5, ignore class 2)': '6_gamma5-lr-2e-5',
    'Opt (Focal gamma=3, include class 2)': '6_gamma3-comcls2',
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1.05], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    ) 

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/felferrari/projects/thesis/experiments/exp_6_wbce_2_8_adamw/results/results_6_wbce_2_8_adamw.data'

In [ ]:
experiments = {
    'Opt (w/out 3d Conv)': 6,
    'Opt (with 3d Conv)': 16,
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1.05], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    ) 

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()